### 1. Dependencies

In [1]:
import torch
from torchvision import transforms
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import argparse
import os
from datetime import datetime
import shutil
import numpy as np

from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
import cv2

from utils_logging import setup_logger

### 2. Choose between Recasens or GazeNet

- Idea is you can just swap 
models.recasens, dataloader.recasens, training.train_recasens, etc...
- with the following
models.gazenet, dataloader.gazenet, training.train_gazenet

In [3]:
from models.face3D import Face3D
from models.__init__ import save_checkpoint, resume_checkpoint
from dataloader.hypo import RetailGaze
from dataloader import chong_imutils
from training.train_chong import train, test, GazeOptimizer

In [4]:
# Logger will save the training and test errors to a .log file 
logger = setup_logger(name='first_logger',
                      log_dir ='./logs/',
                      log_file='train_chong_gooreal.log',
                      log_format = '%(asctime)s %(levelname)s %(message)s',
                      verbose=True)

### 3. Dataloaders
- Choose between GazeDataset (Gazefollow dataset) or GooDataset (GooSynth/GooReal)
- Set paths to image directories and pickle paths. For Gazefollow, images_dir and test_images_dir should be the same and both lead to the path containing the train and test folders.

In [5]:
# Dataloaders for GOO-Synth
batch_size=32
workers=12

images_dir = '/Users/shashimalsenarath/Downloads/RetailGaze_V2-2/'
pickle_path = '/Users/shashimalsenarath/Downloads/RetailGaze_V3_train.pickle'
test_images_dir = '/Users/shashimalsenarath/Downloads/RetailGaze_V2-2/'
test_pickle_path = '/Users/shashimalsenarath/Downloads/RetailGaze_V3_test.pickle'

train_set = RetailGaze(images_dir, pickle_path, 'train')
train_data_loader = DataLoader(dataset=train_set,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=16)

test_set = RetailGaze(test_images_dir, test_pickle_path, 'test')
test_data_loader = DataLoader(test_set, batch_size=batch_size//2,
                            shuffle=False, num_workers=8)

Number of Images: 2745
Number of Images: 589


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [8]:
img, face, head, gt_label, head_box, image_path =  next(iter(train_set))

In [17]:
gt_label

array([0.321375  , 0.28106667])

In [15]:
import matplotlib.pyplot as plt

plt.imshow(img.numpy().transpose(1,2,0))
plt.savefig("image.png")

In [4]:
# Dataloaders for GOO-Real
batch_size=32
workers=12

images_dir = '/home/eee198/Documents/datasets/GOOReal/finalrealdatasetImgsV2/'
pickle_path = '/home/eee198/Documents/datasets/GOOReal/oneshotrealhumans.pickle'
test_images_dir = '/home/eee198/Documents/datasets/GOOReal/finalrealdatasetImgsV2/'
test_pickle_path = '/home/eee198/Documents/datasets/GOOReal/testrealhumans.pickle'

train_set = GooDataset(images_dir, pickle_path, 'train')
train_data_loader = DataLoader(dataset=train_set,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=16)

test_set = GooDataset(test_images_dir, test_pickle_path, 'test')
test_data_loader = DataLoader(test_set, batch_size=batch_size//2,
                            shuffle=False, num_workers=8)

Number of Images: 2451
Number of Images: 2156


In [4]:
# Dataloaders for GAZE

batch_size=32
workers=12
testbatchsize=16

images_dir = '/home/eee198/Documents/datasets/GazeFollowData/'
pickle_path = '/home/eee198/Documents/datasets/GazeFollowData/train_annotations.mat'
test_images_dir = '/home/eee198/Documents/datasets/GazeFollowData/'
test_pickle_path = '/home/eee198/Documents/datasets/GazeFollowData/test_annotations.mat'

train_set = GazeDataset(images_dir, pickle_path, 'train')
train_data_loader = DataLoader(dataset=train_set,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=16)

test_set = GazeDataset(test_images_dir, test_pickle_path, 'test')
test_data_loader = DataLoader(test_set, batch_size=batch_size//2,
                            shuffle=False, num_workers=8)

### 4. Load Model and Set Training Hyperparameters
- For Gazefollow, the model requires the alexnet_places365 pretrained model, provided here: https://urlzs.com/ytKK3
- When resuming training, set to True and set the resume_path for the saved model.
- Here, logging module is initialized (logger) to save training and testing errors.

In [18]:
#!wget https://www.dropbox.com/s/s9y65ajzjz4thve/initial_weights_for_spatial_training.pt
init_weights = 'initial_weights_for_spatial_training.pt'

# Loads model
print("==> Constructing model")
net = Face3D()


# Hyperparameters
start_epoch = 0
max_epoch = 5
learning_rate = 3e-4

# Initial weights chong
print("==> Loading initial weights")
model_dict = net.state_dict()
pretrained_dict = torch.load(init_weights)
pretrained_dict = pretrained_dict['model']
model_dict.update(pretrained_dict)
net.load_state_dict(model_dict)

# Initializes Optimizer
gaze_opt = GazeOptimizer(net, learning_rate)
optimizer = gaze_opt.getOptimizer(start_epoch)

# Resuming Training
resume_training = False
resume_path = './saved_models/chong_goosynth/model_epoch25.pth.tar'
if resume_training:
    net, optimizer, _ = resume_checkpoint(net, optimizer,resume_path)
    test(net, test_data_loader,logger, save_output=True)

==> Constructing model


Using cache found in /Users/shashimalsenarath/.cache/torch/hub/intel-isl_MiDaS_master
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Downloading: "https://github.com/intel-isl/DPT/releases/download/1_0/dpt_hybrid-midas-501f0c75.pt" to /Users/shashimalsenarath/.cache/torch/hub/checkpoints/dpt_hybrid-midas-501f0c75.pt
 33%|███▎      | 155M/470M [01:14<02:31, 2.18MB/s] 


KeyboardInterrupt: 

In [7]:
test(net, test_data_loader,logger, save_output=True)

/home/eee198/anaconda3/envs/gazefollow/lib/python3.7/site-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))
100%|██████████| 135/135 [00:15<00:00,  8.93it/s]
average error: [0.6637493906421568, 0.33433772442945553, 66.62653383895953]


[0.6637493906421568, 0.33433772442945553, 66.62653383895953]

### 5. Training the Model
- Determine in which epochs do you want to save the model, as you might not want to save every epoch
- Training and test errors can be accessed in the logs directory set up earlier

In [8]:
best_l2 = np.inf

for epoch in range(1,5):

    # Update optimizer
    optimizer = gaze_opt.getOptimizer(epoch)

    # Train model
    print('training')
    train(net, train_data_loader, optimizer, epoch, logger)

    # Evaluate model
    #scores = test(net, test_data_loader, logger)
    
    # Save model+optimizer with best L2 Scorehttp://localhost:8888/notebooks/train_chong.ipynb#
    #if scores[1] < best_l2:
    #    best_l2 = scores[1]
    #    save_path = './saved_models/chong_gooreal_notrained/'
    #    save_checkpoint(net, optimizer, 420, save_path)

training


/home/eee198/anaconda3/envs/gazefollow/lib/python3.7/site-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))
100%|██████████| 77/77 [00:38<00:00,  1.99it/s]

training



100%|██████████| 77/77 [00:39<00:00,  1.97it/s]

training



100%|██████████| 77/77 [00:39<00:00,  1.94it/s]

training



100%|██████████| 77/77 [00:39<00:00,  1.96it/s]


In [9]:
test(net, test_data_loader,logger, save_output=True)

100%|██████████| 135/135 [00:16<00:00,  8.39it/s]
average error: [0.8613018095794566, 0.1459625192366771, 26.391563159013245]


[0.8613018095794566, 0.1459625192366771, 26.391563159013245]

In [1]:
import cv2

mask = cv2.imread("/Users/shashimalsenarath/Downloads/RetailGaze_V2-2/1/060433/combined.png",0)

In [2]:
cv2.imshow("mask",mask)